In [4]:

pip install ipython


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Lovely\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [15]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\Lovely\Desktop\AI_Worklab\Lab_Week4_Aug11\lab_natural_language_processing\lab-natural-language-processing\data\kg_train.csv", encoding='latin-1')


# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [43]:

from sklearn.model_selection import train_test_split

# Load the original data
file_path = r"C:\Users\Lovely\Desktop\AI_Worklab\Lab_Week4_Aug11\lab_natural_language_processing\lab-natural-language-processing\data\kg_train.csv"
data = pd.read_csv(file_path, encoding='latin-1')

# Split into train and test sets (e.g., 80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Save test set to a new CSV file
test_path = r"C:\Users\Lovely\Desktop\AI_Worklab\Lab_Week4_Aug11\lab_natural_language_processing\lab-natural-language-processing\data\kg_test.csv"
test_data.to_csv(test_path, index=False, encoding='latin-1')

print("Train/Test split complete.")
print(f"Test data saved to: {test_path}")

train_data['preprocessed_text'] = train_data['text'].apply(clean_text)



Train/Test split complete.
Test data saved to: C:\Users\Lovely\Desktop\AI_Worklab\Lab_Week4_Aug11\lab_natural_language_processing\lab-natural-language-processing\data\kg_test.csv


In [45]:
from sklearn.model_selection import train_test_split

# Step 1: Split data into 80% train+val and 20% test
train_val_data, data_test = train_test_split(data, test_size=0.2, random_state=42)

# Step 2: Split train_val_data into 75% train and 25% val (which is 60% train, 20% val overall)
data_train, data_val = train_test_split(train_val_data, test_size=0.25, random_state=42)

# Optional: Check sizes
print(f"Train size: {data_train.shape}")
print(f"Validation size: {data_val.shape}")
print(f"Test size: {data_test.shape}")

# Step 3: Apply your text cleaning function to all splits if needed
data_train['preprocessed_text'] = data_train['text'].apply(clean_text)
data_val['preprocessed_text'] = data_val['text'].apply(clean_text)
data_test['preprocessed_text'] = data_test['text'].apply(clean_text)


Train size: (3578, 2)
Validation size: (1193, 2)
Test size: (1193, 2)


## Data Preprocessing

In [17]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [19]:
import re

def clean_html(raw_html):
    # 1. Remove inline JavaScript/CSS:
    raw_html = re.sub(r'<(script|style).*?>.*?</\1>', '', raw_html, flags=re.DOTALL | re.IGNORECASE)

    # 2. Remove HTML comments:
    raw_html = re.sub(r'<!--.*?-->', '', raw_html, flags=re.DOTALL)

    # 3. Remove all remaining HTML tags:
    raw_html = re.sub(r'<[^>]+>', '', raw_html)

    # Optional: Replace multiple spaces/newlines with one
    raw_html = re.sub(r'\s+', ' ', raw_html).strip()

    return raw_html


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
import re

def clean_text(text):
    # 1. Remove special characters (keep only letters and space)
    text = re.sub(r'[^A-Za-z\s]', ' ', text)
    
    # 2. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 3. Remove all single characters (e.g., "a", "x", etc.)
    text = re.sub(r'\b\w\b', '', text)

    # 4. Remove single characters from the start (e.g., "aTest" → "Test")
    text = re.sub(r'\b\w{1}\b\s*', '', text)

    # 5. Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # 6. Remove prefixed 'b' if present (from byte strings like b'Text')
    text = re.sub(r'^b\s+', '', text)

    # 7. Convert to lowercase
    text = text.lower()

    # Strip leading/trailing whitespace
    return text.strip()



Index(['text', 'label'], dtype='object')


## Now let's work on removing stopwords
Remove the stopwords.

In [23]:
!pip install nltk






[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Lovely\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return ' '.join(filtered)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lovely\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [25]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # For wordnet data


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lovely\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Lovely\AppData\Roaming\nltk_data...


True

In [26]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)


In [27]:
from nltk.corpus import wordnet
from nltk import pos_tag

def get_wordnet_pos(treebank_tag):
    # Convert Treebank POS tags to WordNet's format
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text_pos(text):
    words = text.split()
    pos_tags = pos_tag(words)
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    return ' '.join(lemmatized_words)


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [28]:
from collections import Counter
import matplotlib.pyplot as plt


In [29]:
def get_top_n_words(corpus, n=None):
    all_words = ' '.join(corpus).split()
    word_freq = Counter(all_words)
    return word_freq.most_common(n)


In [36]:
print(data.columns)


Index(['text', 'label'], dtype='object')


In [37]:
data['clean_text'] = data['text'].apply(clean_text)



In [38]:
# If 0 = ham, 1 = spam (adjust if reversed)
ham_msgs = data[data['label'] == 0]['clean_text']  # or your actual text column name
spam_msgs = data[data['label'] == 1]['clean_text']



In [39]:
top_ham_words = get_top_n_words(ham_msgs, 10)
top_spam_words = get_top_n_words(spam_msgs, 10)

print("Top 10 Ham words:", top_ham_words)
print("Top 10 Spam words:", top_spam_words)


Top 10 Ham words: [('the', 14282), ('to', 8439), ('and', 6836), ('of', 6542), ('in', 5114), ('that', 3247), ('is', 2914), ('for', 2796), ('on', 2700), ('you', 2033)]
Top 10 Spam words: [('the', 39757), ('to', 31509), ('of', 27934), ('and', 21762), ('in', 17976), ('you', 17941), ('this', 14583), ('my', 12159), ('your', 12029), ('for', 11837)]


## Extra features

In [46]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train_data['money_mark'] = train_data['preprocessed_text'].str.contains(money_simbol_list)*1
train_data['suspicious_words'] = train_data['preprocessed_text'].str.contains(suspicious_words)*1
train_data['text_len'] = train_data['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

train_data.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
3821,back -- we need to talkCall only my berry.,0,back we need to talkcall only my berry,0,0,38
2407,What is mullen's first name?,0,what is mullen first name,0,0,25
1173,Greetings from Dubai=2CThis letter must come t...,1,greetings from dubai cthis letter must come to...,1,1,3657
4231,Sullivan Jacob J <SullivanJJ@state.gov>Friday ...,0,sullivan jacob sullivanjj state gov friday dec...,0,1,596
1666,Fyi,0,fyi,0,0,3


## How would work the Bag of Words with Count Vectorizer concept?

It’s a way to convert text into numbers so a machine learning model can understand it.

Each unique word in your entire text corpus is treated as a feature (a column).

For each document (like an email or sentence), you count how many times each word appears.

The result is a matrix where:

Rows = documents

Columns = unique words (vocabulary)

Values = counts of words in each document

In [47]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample data - list of documents (strings)
documents = [
    "I love AI and machine learning",
    "Machine learning is fun",
    "AI is the future"
]

# Create the vectorizer
vectorizer = CountVectorizer()

# Fit on documents and transform text to BoW counts
bow_matrix = vectorizer.fit_transform(documents)

# Convert to array for visualization
print(bow_matrix.toarray())

# Get feature names (words)
print(vectorizer.get_feature_names_out())


[[1 1 0 0 0 1 1 1 0]
 [0 0 1 0 1 1 0 1 0]
 [1 0 0 1 1 0 0 0 1]]
['ai' 'and' 'fun' 'future' 'is' 'learning' 'love' 'machine' 'the']


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assume your dataset text column is called 'preprocessed_text'
# For example, on your training data:
texts = data_train['preprocessed_text'].tolist()

# Step 1: Load the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Step 2: Fit and transform your dataset
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

# Step 3: Print the shape of the vectorized dataset
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")


TF-IDF matrix shape: (3578, 77348)


fit_transform() learns the vocabulary and IDF from texts, then transforms texts into a TF-IDF-weighted sparse matrix.

The .shape prints (number_of_documents, number_of_unique_words)

## And the Train a Classifier?

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Assume you already have:
# data_train and data_val splits
# TF-IDF vectorizer fitted on train data (tfidf_vectorizer)

# 1. Vectorize train and validation text
X_train = tfidf_vectorizer.transform(data_train['preprocessed_text'])
X_val = tfidf_vectorizer.transform(data_val['preprocessed_text'])

# 2. Get labels
y_train = data_train['label']
y_val = data_val['label']

# 3. Initialize the classifier
clf = LogisticRegression(max_iter=1000)

# 4. Train the classifier
clf.fit(X_train, y_train)

# 5. Predict on validation set
y_pred = clf.predict(X_val)

# 6. Evaluate
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))


Accuracy: 0.9815590947191953
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       670
           1       1.00      0.96      0.98       523

    accuracy                           0.98      1193
   macro avg       0.98      0.98      0.98      1193
weighted avg       0.98      0.98      0.98      1193



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [67]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Download once
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# Load data
import pandas as pd

train_data = pd.read_csv(
    r"C:\Users\Lovely\Desktop\AI_Worklab\Lab_Week4_Aug11\lab_extra_kaggle_ham_or_spam\kg_train.csv",
    encoding='latin-1'
)



# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', ' ', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    tokens = nltk.word_tokenize(text)
    
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words and len(w) > 1]
    return ' '.join(tokens)

train_data['clean_text'] = train_data['text'].astype(str).apply(preprocess_text)

# Optional extra features
def add_extra_features(df):
    df['text_len'] = df['text'].apply(len)
    df['money_mark'] = df['text'].apply(lambda x: 1 if '$' in x or '£' in x else 0)
    suspicious = ['free', 'win', 'winner', 'cash', 'urgent']
    df['suspicious_words'] = df['text'].apply(lambda x: sum([1 for word in suspicious if word in x.lower()]))
    return df

train_data = add_extra_features(train_data)

# Split for evaluation
X_train, X_val, y_train, y_val = train_test_split(train_data, train_data['label'], test_size=0.2, random_state=42)

# Vectorizers
count_vec = CountVectorizer()
tfidf_vec = TfidfVectorizer()

# --- A. Bag of Words Only ---
X_bow = count_vec.fit_transform(X_train['clean_text'])
X_val_bow = count_vec.transform(X_val['clean_text'])

model_bow = MultinomialNB()
model_bow.fit(X_bow, y_train)
pred_bow = model_bow.predict(X_val_bow)
acc_bow = accuracy_score(y_val, pred_bow)

# --- B. TF-IDF Only ---
X_tfidf = tfidf_vec.fit_transform(X_train['clean_text'])
X_val_tfidf = tfidf_vec.transform(X_val['clean_text'])

model_tfidf = MultinomialNB()
model_tfidf.fit(X_tfidf, y_train)
pred_tfidf = model_tfidf.predict(X_val_tfidf)
acc_tfidf = accuracy_score(y_val, pred_tfidf)

# --- C. Bag of Words + Extra Features ---
X_bow_extra = pd.concat([
    pd.DataFrame(X_bow.toarray()),
    X_train[['text_len', 'money_mark', 'suspicious_words']].reset_index(drop=True)
], axis=1)
X_val_bow_extra = pd.concat([
    pd.DataFrame(X_val_bow.toarray()),
    X_val[['text_len', 'money_mark', 'suspicious_words']].reset_index(drop=True)
], axis=1)

model_bow_extra = MultinomialNB()
model_bow_extra.fit(X_bow_extra, y_train)
pred_bow_extra = model_bow_extra.predict(X_val_bow_extra)
acc_bow_extra = accuracy_score(y_val, pred_bow_extra)

# --- D. TF-IDF + Extra Features ---
X_tfidf_extra = pd.concat([
    pd.DataFrame(X_tfidf.toarray()),
    X_train[['text_len', 'money_mark', 'suspicious_words']].reset_index(drop=True)
], axis=1)
X_val_tfidf_extra = pd.concat([
    pd.DataFrame(X_val_tfidf.toarray()),
    X_val[['text_len', 'money_mark', 'suspicious_words']].reset_index(drop=True)
], axis=1)

model_tfidf_extra = MultinomialNB()
model_tfidf_extra.fit(X_tfidf_extra, y_train)
pred_tfidf_extra = model_tfidf_extra.predict(X_val_tfidf_extra)
acc_tfidf_extra = accuracy_score(y_val, pred_tfidf_extra)

# --- RESULTS ---
print("📊 Accuracy Comparison:")
print(f"Bag of Words only: {acc_bow:.4f}")
print(f"TF-IDF only: {acc_tfidf:.4f}")
print(f"BoW + extra features: {acc_bow_extra:.4f}")
print(f"TF-IDF + extra features: {acc_tfidf_extra:.4f}")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Lovely\\Desktop\\AI_Worklab\\Lab_Week4_Aug11\\lab_extra_kaggle_ham_or_spam\\kg_train.csv'